# [튜토리얼1] 정형 데이터 다루기

이 튜토리얼은 **정형 데이터(structured data)** 를 다루는 방법을 소개합니다. 

정형 데이터란 구조화된 데이터를 의미하며,예를 들어 CSV에서 읽은 표 형식의 데이터가 이에 해당합니다. 정형 데이터를 다루기 위해 [케라스(Keras) API](https://www.tensorflow.org/guide/keras)를 사용하여 모델을 정의하고 [피쳐 열](https://www.tensorflow.org/api_docs/python/tf/feature_column)(feature column)을 사용하여 CSV의 열을 모델 훈련에 필요한 피쳐로 매핑하겠습니다. 

이 튜토리얼은 다음 내용을 포함합니다:

* [판다스](https://pandas.pydata.org/)(Pandas)를 사용하여 CSV 파일을 읽기
* [tf.data](https://www.tensorflow.org/guide/datasets)를 사용하여 행을 섞고 배치로 나누는 입력 파이프라인(pipeline) 만들기
* CSV의 열을 피쳐 열(`feature_column`)을 사용해 모델 훈련에 필요한 피쳐로 매핑하기
* 케라스(Keras)를 사용하여 모델 구축, 훈련, 평가하기

In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# 목차
1. 데이터셋
2. 판다스로 데이터프레임 만들기
3. 데이터프레임을 훈련 세트, 검증 세트, 테스트 세트로 나누기
4. tf.data를 사용하여 입력 파이프라인 만들기
5. 입력 파이프라인 이해하기
6. 여러 종류의 피쳐 열 알아보기
7. 사용할 열 선택하기
8. 모델 생성, 컴파일, 훈련

## 1. 데이터셋

클리블랜드(Cleveland) 심장병 재단에서 제공한 작은 [CSV 데이터셋](https://archive.ics.uci.edu/ml/datasets/heart+Disease)을 사용하겠습니다. 이 CSV 파일은 수백 개의 행으로 이루어져 있습니다. 각 행은 환자 한 명을 나타내고 각 열은 해당 환자에 대한 속성 값을 나타냅니다. 이 정보를 사용해 환자의 심장병 발병 여부를 예측해 보겠습니다. 즉 이 데이터셋은 이진 분류 문제입니다.

다음은 데이터셋에 대한 [설명](https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names)입니다. 데이터셋 내에 **수치형과 범주형 열**이 모두 있다는 점을 주목하세요.

>열| <center>설명</center>| 피쳐 타입 | 데이터 타입
>------------|: -------------------- :|----------------------|-----------------
>Age | <center>나이</center> | 수치형 | 정수
>Sex | <center>성별(1 = 남성; 0 = 여성)</center> | 범주형 | 정수
>CP | <center>가슴 통증 유형 (0, 1, 2, 3, 4)</center> | 범주형 | 정수
>Trestbpd | <center>안정 혈압 (병원 입원시 mm Hg)</center> | 수치형 | 정수
>Chol | <center>혈청 콜레스테롤 (mg/dl)</center> | 수치형 | 정수
>FBS | <center>공복 혈당 > 120 mg/dl (1 = true; 0 = false)</center> | 범주형 | 정수
>RestECG | <center>안정 심전도 결과 (0, 1, 2)</center> | 범주형 | 정수
>Thalach | <center>최대 심박동수</center> | 수치형 | 정수
>Exang | <center>협심증 유발 운동 (1 = yes; 0 = no)</center> | 범주형 | 정수
>Oldpeak | <center>비교적 안정되기까지 운동으로 유발되는 ST depression</center> | 수치형 | 정수
>Slope | <center>최대 운동 ST segment의 기울기</center> | 수치형 | 실수
>CA | <center>형광 투시된 주요 혈관의 수 (0-3)</center> | 수치형 | 정수
>Thal | <center>심장에 혈류를 측정하는 Thallium stress test 결과, normal = 보통; fixed	 = 해결된 결함; reversible	 = 해결가능한 결함</center> | 범주형 | 문자열
>Target | <center>심장병 진단 (1 = true; 0 = false)</center> | 분류 | 정수

## 2. 판다스로 데이터프레임 만들기

[판다스](https://pandas.pydata.org/)는 정형 데이터를 읽고 조작하는데 유용한 유틸리티 함수를 많이 제공하는 파이썬 라이브러리입니다. 판다스를 이용해 URL로부터 데이터를 다운로드하여 읽어 데이터프레임 형식의 데이터로 변환하겠습니다.

In [ ]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

## 3. 데이터프레임을 훈련 세트, 검증 세트, 테스트 세트로 나누기

하나의 CSV 파일에서 데이터셋을 다운로드했습니다. 이를 **훈련 세트, 검증 세트, 테스트 세트**로 나누겠습니다.

훈련 세트는 전체 데이터 셋 중 80%, 테스트 세트는 전체 데이터 셋 중 20%를 사용하며, 검증 세트는 나눈 훈련 세트의 20%를 사용합니다.

In [ ]:
print("데이터프레임의 shape : ",dataframe.shape)
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print('훈련 샘플 : %d개' %len(train))
print('검증 샘플 : %d개' %len(val))
print('테스트 샘플 : %d개' %len(test))

## 4. tf.data를 사용하여 입력 파이프라인 만들기

그다음 [tf.data](https://www.tensorflow.org/guide/datasets)를 사용하여 데이터프레임을 감싸겠습니다. 이렇게 하면 피쳐 열(`feature_column`)을 사용하여 판다스 데이터프레임의 열을 모델 훈련에 필요한 피쳐로 **매핑**할 수 있습니다. 만약 아주 큰 CSV 파일(메모리에 들어갈 수 없을 정도로 큰 파일)을 다룬다면 tf.data로 디스크 디렉토리에서 데이터를 읽어올 수 있습니다. 그러나 이번 튜토리얼에서 사용하는 CSV 파일은 303개의 행을 가진 작은 파일이기 때문에 디스크 디렉토리에서 읽어올 필요가 없습니다.    

In [ ]:
# 판다스 데이터프레임으로부터 tf.data 데이터셋을 만들기 위한 함수
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
batch_size = 5 # 예제를 위해 작은 배치 크기를 사용합니다.
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## 5. 입력 파이프라인 이해하기

앞서 만든 입력 파이프라인을 호출하여 반환되는 데이터 포맷을 확인해 보겠습니다. 간단한 출력을 위해 작은 배치 크기를 사용합니다.

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
    print('전체 피쳐:', list(feature_batch.keys()))
    print('나이 피쳐의 배치:', feature_batch['age'])
    print('타깃의 배치:', label_batch )

이 데이터셋은 데이터프레임의 열 이름을 키로 갖는 딕셔너리를 반환합니다. 딕셔너리의 각 키(key)에는 데이터프레임 열의 값이 매핑되어 있습니다.

## 6. 여러 종류의 피쳐 열(feature_column) 알아 보기

텐서플로는 여러 종류의 피쳐 열을 제공합니다. 다음과 같이 몇 가지 피쳐 열을 만들어서 데이터프레임의 열을 변환하는 방법을 알아 보겠습니다.

In [ ]:
# 피쳐 열을 시험해 보기 위해 샘플 배치를 만듭니다.
example_batch = next(iter(train_ds))[0]

In [ ]:
# 피쳐 열을 만들고 배치 데이터를 변환하는 함수
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

### 6.1 수치형 열

피쳐 열의 출력은 모델의 입력이 됩니다. 앞서 정의한 함수를 사용하여 데이터프레임의 각 열이 어떻게 변환되는지 알아보겠습니다. [수치형 열](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column)은 가장 간단한 종류의 열로, 이 열은 **실수 피쳐**를 표현하는데 사용됩니다. 이 열을 사용하면 모델은 데이터프레임 열의 값을 변형시키지 않고 그대로 전달 받습니다.

In [ ]:
age = feature_column.numeric_column("age")
demo(age)

심장병 데이터셋 데이터프레임의 대부분 열은 이러한 수치형 열 입니다.

### 6.2 버킷형 열

종종 모델에 수치 값을 바로 주입하는 대신 수치 값의 구간을 나누어 이를 기반으로 **범주형**으로 변환할 필요가 있습니다. 원본 데이터가 사람의 나이를 표현한다고 가정해 보죠. 나이를 수치형 열로 표현하는 대신 [버킷형 열](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column)(bucketized column)을 사용하여 나이를 몇 개의 버킷(bucket)으로 분할할 수 있습니다. 다음의 원-핫 인코딩(one-hot encoding)된 값은 각 열이 매칭되는 나이 범위를 나타냅니다.

In [ ]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

### 6.3 범주형 열

이 데이터셋에서 `thal` 열은 문자열입니다(예를 들어 'fixed', 'normal', 'reversible'). 문자열은 모델에 바로 주입할 수 없기 때문에 대신 문자열을 먼저 수치형으로 매핑해야 합니다. 범주형 열(categorical column)을 사용하여 문자열을 원-핫 벡터로 표현할 수 있습니다. 문자열 목록은 [`categorical_column_with_vocabulary_list`](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list)를 사용하여 리스트로 전달하거나 [`categorical_column_with_vocabulary_file`](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_file)을 사용하여 파일에서 읽을 수 있습니다.

In [ ]:
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

더 복잡한 데이터셋에는 범주형(예를 들면 문자열)인 열이 많을 수 있습니다. 피쳐 열은 특히 이러한 범주형 데이터를 다룰 때 진가가 발휘됩니다. 이 데이터셋에는 범주형 열이 하나 뿐이지만 다른 데이터셋에서 사용할 수 있는 여러 종류의 피쳐 열을 소개하겠습니다.

### 6.4 임베딩 열

가능한 문자열이 몇 개가 있는 것이 아니라 범주마다 수천 개 이상의 값이 있는 경우를 상상해 보겠습니다. 여러 가지 이유로 범주의 개수가 늘어남에 따라 원-핫 인코딩으로 변환하게 된다면, 고차원 [희소 행렬(sparse matrix)](https://ko.wikipedia.org/wiki/%ED%9D%AC%EC%86%8C%ED%96%89%EB%A0%AC)이 되어 신경망을 잘 훈련시키기 어려워집니다. 그러나 **임베딩 열(embedding column)** 을 사용하면 이런 제한을 극복할 수 있습니다. 고차원 원-핫 벡터로 데이터를 표현하는 대신 [임베딩 열](https://www.tensorflow.org/api_docs/python/tf/feature_column/embedding_column)을 사용하여 저차원으로 데이터를 표현합니다. 이 벡터는 0 또는 1이 아니라 각 원소에 어떤 숫자도 넣을 수 있는 밀집 벡터(dense vector)입니다. 임베딩의 크기(아래 예제에서는 8입니다)는 튜닝 대상 파라미터입니다.

- 핵심 포인트: 범주형 열에 가능한 값이 많을 때는 임베딩 열을 사용하는 것이 최선입니다. 여기에서는 예시를 목적으로 하나를 사용하지만 나중에 다른 데이터셋에도 수정하여 적용할 수 있습니다.

In [ ]:
# 임베딩 열의 입력은 앞서 만든 범주형 열입니다.
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

### 6.5 해시 피쳐 열

가능한 값이 많은 범주형 열을 표현하는 또 다른 방법은 [`categorical_column_with_hash_bucket`](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_hash_bucket)을 사용하는 것입니다. 이 피쳐 열은 입력의 해시(hash) 값을 계산한 다음 `hash_bucket_size` 크기의 버킷 중 하나를 선택하여 문자열을 인코딩합니다. 이 열을 사용할 때는 어휘 목록을 제공할 필요가 없고 공간을 절약하기 위해 실제 범주의 개수보다 훨씬 작게 해시 버킷(bucket)의 크기를 정할 수 있습니다.

- 핵심 포인트: 이 기법의 큰 단점은 다른 문자열이 같은 버킷에 매핑될 수 있다는 것입니다. 그럼에도 실전에서는 일부 데이터셋에서 잘 작동합니다.

In [ ]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

### 6.6 교차 피쳐 열

여러 피쳐를 연결하여 하나의 피쳐로 만드는 것을 교차 피쳐(feature cross)이라고 합니다. 교차 피쳐를 통해 모델이 피쳐의 조합에 대한 가중치를 학습할 수 있습니다. 이 예제에서는 `age`와 `thal`의 교차 피쳐를 만들어 보겠습니다. `crossed_column`은 모든 가능한 조합에 대한 해시 테이블을 만들지 않고 `hashed_column` 매개변수를 사용하여 해시 테이블의 크기를 선택합니다.

In [ ]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

## 7. 사용할 열 선택하기

여러 가지 피쳐 열을 사용하는 방법을 보았으므로 이제 이를 사용하여 모델을 훈련하겠습니다. 이 튜토리얼의 목적은 피쳐 열을 사용하는 완전한 코드(예를 들면 작동 방식)를 제시하는 것이므로 임의로 몇 개의 열을 선택하여 모델을 훈련하겠습니다.

- 핵심 포인트: 제대로 된 모델을 만들어야 한다면 대용량의 데이터셋을 사용하고 어떤 피쳐를 포함하는 것이 가장 의미있는지, 또 어떻게 표현해야 할지 신중하게 생각하세요.

In [ ]:
feature_columns = []

# 수치형 열
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))

# 버킷형 열
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# 범주형 열
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# 임베딩 열
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# 교차 피쳐 열
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

### 7.1 피쳐 레이어 만들기

피쳐 열을 정의하고 나면 [DenseFeatures](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/DenseFeatures) 레이어를 사용해 케라스 모델에 주입할 수 있습니다.

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

앞서 피쳐 열의 작동 예를 보이기 위해 작은 배치 크기(5)를 사용했습니다. 여기에서는 조금 더 큰 배치 크기(32)로 입력 파이프라인을 만듭니다.

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## 8. 모델 생성, 컴파일, 훈련

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("정확도", accuracy)

- 핵심 포인트: 일반적으로 크고 복잡한 데이터셋일 경우 딥러닝 모델에서 최선의 결과를 얻습니다. 이런 작은 데이터셋에서는 기본 모델로 **결정 트리(decision tree)** 나 **랜덤 포레스트(random forest)** 를 사용하는 것이 권장됩니다. 이 튜토리얼의 목적은 정확한 모델을 훈련하는 것이 아니라 정형 데이터를 다루는 방식을 설명하는 것입니다. 실전 데이터셋을 다룰 때 이 코드를 시작점으로 사용하세요.

# Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.